# Подготовка датасета для обучения

Мультимодальные данные — это информация из разных источников в разных форматах. В машинном обучении под мультимодальностью обычно понимают комбинацию как минимум двух типов данных (модальностей): текста, изображений, аудио или видео. 

Мультимодальные системы умеют извлекать и комбинировать признаки из разных модальностей, что помогает делать более точные предсказания по сравнению с унимодальными подходами.

Особенно интересны задачи кросс-модального поиска, где нужно находить соответствия между разными модальностями. Например, поиск изображений по текстовому запросу или генерация текстовых описаний по изображению. 

При работе с мультимодальными данными можно столкнуться с несколькими сложностями:

- Разные модальности имеют разную природу и структуру. Например, изображения представляют собой регулярные сетки пикселей, а текст — это последовательность символов или слов.
    
- Модальности могут иметь разную информативность для конкретной задачи. В некоторых случаях одна модальность может доминировать, а другая — лишь незначительно улучшать результаты.
    
- Возникает проблема согласования временных или пространственных масштабов, особенно при работе с видео и аудио.

Напишем базовый загрузчик данных для обучения мультимодальной модели, работающей с текстами и картинками.

In [1]:
from ml_dl_experiments import settings

root_dataset_path: str = settings.SOURCE_PATH + "datasets/multimodal_practice/"

In [4]:
from functools import partial

from torch.utils.data import Dataset, DataLoader
import timm
from transformers import AutoTokenizer
import torchvision.transforms.v2 as T 
from torchvision.transforms import ToTensor
from PIL import Image
import albumentations as A


class MultimodalDataset(Dataset):
    def __init__(self, df, text_model, image_model, transforms):
        self.df = df
        self.image_cfg = timm.get_pretrained_cfg(image_model)
        self.tokenizer = AutoTokenizer.from_pretrained(text_model)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, "text"]
        label = self.df.loc[idx, "label"]

        img_path = self.df.loc[idx, "image_path"]
        image = Image.open(f"data/images/{img_path}").convert('RGB')
        image = self.transforms(image=np.array(image))["image"]

        return {"label": label, "image": image, "text": text}


def collate_fn(batch, tokenizer):
    texts = [item["text"] for item in batch]
    images = torch.stack([item["image"] for item in batch])
    labels = torch.LongTensor([item["label"] for item in batch])

    tokenized_input = tokenizer(texts,
                                return_tensors="pt",
                                padding="max_length",
                                truncation=True)
    return {
        "label": labels,
        "image": images,
        "input_ids": tokenized_input["input_ids"],
        "attention_mask": tokenized_input["attention_mask"]
    }

Однако этому датасету не хватает важной составляющей для обучения — аугментаций. Также токенизация текстов в датасете выполняется по отдельности, поэтому имеет смысл вынести всю обработку в collate_fn, которую можно затем передать в DataLoader.

In [5]:
text_model = "bert-base-uncased"
image_model = 'tf_efficientnet_b0'
tokenizer = AutoTokenizer.from_pretrained(text_model)
cfg = timm.get_pretrained_cfg(image_model)

transforms = A.Compose(
    [
        A.SmallestMaxSize(max_size=max(cfg.input_size[1], cfg.input_size[2]),
                          p=1.0),
        A.RandomCrop(height=cfg.input_size[1], width=cfg.input_size[2], p=1.0),
        A.Affine(scale=(0.8, 1.2),
                 rotate=(-15, 15),
                 translate_percent=(-0.1, 0.1),
                 shear=(-10, 10),
                 fill=0,
                 p=0.8),
        A.CoarseDropout(num_holes_range=(2, 8),
                        hole_height_range=(int(0.07 * cfg.input_size[1]),
                                           int(0.15 * cfg.input_size[1])),
                        hole_width_range=(int(0.1 * cfg.input_size[2]),
                                          int(0.15 * cfg.input_size[2])),
                        fill=0,
                        p=0.5),
        A.ColorJitter(
            brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.7),
        A.Normalize(mean=cfg.mean, std=cfg.std)
    ]
)

In [7]:
import pandas as pd

df = pd.read_csv(root_dataset_path+"items.csv")
ds = MultimodalDataset(df,
                       text_model=text_model,
                       image_model=image_model,
                       transforms=transforms)

loader = DataLoader(ds,
                    batch_size=1,
                    shuffle=False,
                    collate_fn=partial(collate_fn, tokenizer=tokenizer)) 

## Текстовые аугментации
Они служат тем же целям, что картиночные, то есть помогают разнообразить выборку и улучшить сходимость модели. Их можно разделить на следующие группы:

- Замена символов — случайные замены/перестановки символов в словах для имитации опечаток.
    
- Замена слов — случайные перестановки соседних слов/предложений или замена слов на синонимы.

- Обратный перевод — целый текст переводится на какой-то другой язык с помощью сторонней модели, а затем обратно на исходный.

Такие текстовые аугментации в некоторых случаях могут заметно обогатить изначально небольшой датасет без особых дополнительных усилий по поиску новых данных. Но и ограничения у них есть, ведь если использовать их слишком часто, они могут сильно исказить исходный смысл предложений.

Текстовые аугментации можно вычислять как в момент обучения с целью экономии памяти, так и непосредственно в офлайне перед обучением. Последнее особенно актуально, если используется только один вид текстовых аугментаций (например, обратный перевод).

Текстовые аугментации опциональны и менее популярны, чем аугментации картинок. Но их стоит иметь в виду, ведь иногда они могут помочь дотянуть пару процентных пунктов к целевой метрике.

# Реализация архитектуры мультимодальной нейросети

## Архитектура мультимодальной сети

При разработке мультимодальной архитектуры нужно учитывать два основных момента: объединение энкодеров/декодеров разных модальностей и объединение эмбеддингов разных модальностей.
Чтобы объединить две модели в одну, достаточно:

- Указать их в рамках одного класса nn.Module и проинициализировать.
    
- В методе forward() рассчитать эмбеддинги для каждой модели.

In [19]:
from transformers import AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer

import torch
import torch.nn as nn
import timm


class BaseDefaultMultimodalModel(nn.Module):
    def __init__(self,
                 text_model_name='bert-base-uncased',
                 image_model_name='resnet50'):
        super().__init__()

        # Текстовая ветка
        self.text_model = AutoModel.from_pretrained(text_model_name)

        # Визуальная ветка
        self.image_model = timm.create_model(
            image_model_name,
            pretrained=True,
            num_classes=0  # Возвращаем вектор признаков, а не классы
        )

    def forward(self, text_input, image_input):
        # эмбеддинги текста
        text_features = self.text_model(**text_input).last_hidden_state[:,  0, :]

        # эмбеддинги изображений
        image_features = self.image_model(image_input)
        return text_features, image_features

text_model = 'bert-base-uncased'
image_model = 'resnet50'
m = BaseDefaultMultimodalModel(text_model_name=text_model,
                        image_model_name=image_model)

# 2 примера текста и картинки для инференса
tk = AutoTokenizer.from_pretrained(text_model)
tokenized = tk(["text", "text2"],
               return_tensors="pt",
               padding='max_length',
               truncation=True)


# вместо самой картинки используем случайный тензор правильной размерности
# размерность тензора берём из конфига и распаковываем через `*`
img = torch.randn(2, *m.image_model.pretrained_cfg["input_size"])

t, i = m(tokenized, img)
t.shape, i.shape 

(torch.Size([2, 768]), torch.Size([2, 2048]))

## Объединение эмбеддингов

Варианты объединения: 

- После расчёта эмбеддингов по каждой модальности конкатенируем их, а поверх используем классификационную голову (например, MLP-слой после).
    
- Проделаем то же самое, что и выше, но поэлементно перемножаем эмбеддинги.
    
- Кросс-модальное внимание.

Во всех случаях исходные эмбеддинги из моделей, как правило, приводятся к одной, меньшей размерности. Это решает две задачи:

- Позволяет пропорционально учитывать каждую из модальностей в дальнейшем предсказании.
    
- Способствует более устойчивому обучению — в случае больших размерностей эмбеддингов модели часто переобучаются.

Выбор между конкретным вариантом часто основывается на типе и сложности задачи: если в задаче важно взаимодополнение модальностей, используют кросс-модальное внимание, в случае обычной классификации иногда достаточно перемножения/конкатенации.

Давайте посмотрим на реализацию слияния эмбеддингов через поэлементное перемножение. Для этого адаптируем код BaseMultimodalModel, добавив в него проекции эмбеддинга каждой модальности в пространство меньшей размерности — 256 — и визуализируем их:

In [ ]:
class BaseMultiplyMultimodalModel(nn.Module):
    def __init__(self,
                 text_model_name='bert-base-uncased',
                 image_model_name='resnet50'):
        super().__init__()

        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.image_model = timm.create_model(
            image_model_name,
            pretrained=True,
            num_classes=0 
        )

        # проецируем каждый из векторов в размерность 256
        # входную размерность линейного слоя задаём динамически из конфигов моделей
        self.text_proj = nn.Linear(self.text_model.config.hidden_size, 256)
        self.image_proj = nn.Linear(self.image_model.num_features, 256)

    def forward(self, text_input, image_input):
        text_features = self.text_model(**text_input).last_hidden_state[:,  0, :]
        image_features = self.image_model(image_input)

        # вычисляем проекции эмбеддингов и объединяем их
        text_emb = self.text_proj(text_features)
        image_emb = self.image_proj(image_features)
        
        print("Размерности проекций эмбеддингов:",text_emb.shape, image_emb.shape)

        fused_emb = text_emb * image_emb
        return fused_emb

text_model = 'bert-base-uncased'
image_model = 'resnet50'
m = BaseMultiplyMultimodalModel(text_model_name=text_model,
                        image_model_name=image_model)

# 2 примера текста и картинки для инференса
tk = AutoTokenizer.from_pretrained(text_model)
tokenized = tk(["text", "text2"],
               return_tensors="pt",
               padding='max_length',
               truncation=True)

img = torch.randn(2, *m.image_model.pretrained_cfg["input_size"])

emb = m(tokenized, img)
emb.shape 

Размерности проекций эмбеддингов: torch.Size([2, 256]) torch.Size([2, 256])


torch.Size([2, 256])

Вы получили каркас для решения мультимодальной задачи — эмбеддинги спроецированы в одно пространство и объединены. Как правило, для проецирования используют числа меньшей размерности, чем каждый из эмбеддингов модальностей, являющиеся степенью двойки (256, 512, 1024, ..). Однако могут быть и другие значения, которые можно многократно разделить на 2 (например, 768). Это связано с особенностями хранения и обработки тензоров на GPU — такие размеры позволяют эффективнее использовать память и вычислительные ресурсы.

Адаптируем класс модели, эмбеддинги сливаются через конкатинацию:

In [12]:
from transformers import AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer

import torch
import torch.nn as nn

import timm


class BaseConcatMultimodalModel(nn.Module):
    def __init__(self,
                 text_model_name='bert-base-uncased',
                 image_model_name='resnet50',
                 emb_dim=256):
        super().__init__()
        self.emb_dim = emb_dim
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.image_model = timm.create_model(
            image_model_name,
            pretrained=True,
            num_classes=0 
        )

        self.text_proj = nn.Linear(self.text_model.config.hidden_size, emb_dim)
        self.image_proj = nn.Linear(self.image_model.num_features, emb_dim)

    def forward(self, text_input, image_input):
        text_features = self.text_model(**text_input).last_hidden_state[:,  0, :]
        image_features = self.image_model(image_input)

        text_emb = self.text_proj(text_features)
        image_emb = self.image_proj(image_features)

        fused_emb = torch.cat([text_emb, image_emb], dim=1)
        return fused_emb


text_model = 'bert-base-uncased'
image_model = 'resnet50'
m = BaseConcatMultimodalModel(text_model_name=text_model,
                        image_model_name=image_model)


# 2 примера текста и картинки для инференса
tk = AutoTokenizer.from_pretrained(text_model)
tokenized = tk(["text", "text2"],
               return_tensors="pt",
               padding='max_length',
               truncation=True)

img = torch.randn(2, *m.image_model.pretrained_cfg["input_size"])
emb = m(tokenized, img)
emb.shape 

torch.Size([2, 512])

## Крос-модальное внимание:

В самом простом варианте реализации кросс-модальное внимание — это просто attention-слой, в котором в качестве запросов выступают значения одной модальности, а в качестве ключей и значений — другой. 

Есть и другие варианты реализации, но мы остановимся на базовом.


Посмотрите на него в коде, в котором BaseMultimodalModel используется как часть общей модели. Инициализируем и добавим слой MultiheadAttention, в который передадим полученные эмбеддинги картинок и текстов:

In [20]:
class BaseMultimodalModel(nn.Module):
    def __init__(self,
                 text_model_name='bert-base-uncased',
                 image_model_name='resnet50',
                 emb_dim=256):
        super().__init__()
        self.emb_dim = emb_dim
        self.text_model = AutoModel.from_pretrained(text_model_name)
        self.image_model = timm.create_model(
            image_model_name,
            pretrained=True,
            num_classes=0 
        )

        # проецируем каждый из векторов в размерность 256
        # входную размерность линейного слоя задаём динамически из конфигов моделей
        self.text_proj = nn.Linear(self.text_model.config.hidden_size, emb_dim)
        self.image_proj = nn.Linear(self.image_model.num_features, emb_dim)

    def forward(self, text_input, image_input):
        text_features = self.text_model(**text_input).last_hidden_state[:,  0, :]
        image_features = self.image_model(image_input)

        # вычисляем проекции эмбеддингов и объединяем их
        text_emb = self.text_proj(text_features)
        image_emb = self.image_proj(image_features)
        
        print("Размерности проекций эмбеддингов:",text_emb.shape, image_emb.shape)

        return text_emb, image_emb

In [21]:
import torch.nn as nn


class CrossAttentionModel(nn.Module):
    def __init__(self, text_model_name='bert-base-uncased', image_model_name='resnet50'):
        super().__init__()
        # Инициализация базовых моделей
        self.base_model = BaseMultimodalModel(text_model_name, image_model_name)
        
        # Механизм внимания
        self.cross_attn = nn.MultiheadAttention(
            embed_dim=self.base_model.emb_dim, 
            num_heads=4)
        
    def forward(self, text_input, image_input):
        # Получаем эмбеддинги модальностей
        text_emb, image_emb = self.base_model(text_input, image_input)
        
        # Подготовка для внимания - добавляем размерность последовательности
        # так как она необходима для вычислений в MultiheadAttention
        text_emb = text_emb.unsqueeze(0)  # [1, batch_size, emb_dim]
        image_emb = image_emb.unsqueeze(0)  # [1, batch_size, emb_dim]
        
        # Текст как запрос, изображение как ключ/значение
        attended_emb, _ = self.cross_attn(
            query=text_emb,
            key=image_emb,
            value=image_emb
        )
        
        # Возвращаем эмбеддинги без первой размерности
        # так как она нам нужна только в шаге с расчётом attetion
        return attended_emb.squeeze(0)


cam = CrossAttentionModel()
out = cam(tokenized, img)
out.shape 

Размерности проекций эмбеддингов: torch.Size([2, 256]) torch.Size([2, 256])


torch.Size([2, 256])

Добавьте в архитектуру CrossAttentionModel классификационный слой — двухслойный MLP с уменьшением размерности в два раза. Используйте дропаут и нормализации. Метод forward() должен возвращать логиты предсказаний. Количество классов num_classes установите равным 2.

In [1]:
import torch.nn as nn


class CrossAttentionModel(nn.Module):
    def __init__(self, 
                 text_model_name='bert-base-uncased', 
                 image_model_name='resnet50', 
                 num_classes=2):
        super().__init__()
        self.base_model = BaseMultimodalModel(text_model_name, image_model_name)
        emb_dim = self.base_model.emb_dim
        
        self.cross_attn = nn.MultiheadAttention(
            embed_dim=emb_dim, 
            num_heads=4)

        self.classifier = nn.Sequential(
            nn.Linear(emb_dim, emb_dim // 2),    # Уменьшение размерности
            nn.LayerNorm(emb_dim // 2),          # Нормализация
            nn.ReLU(),                           # Активация
            nn.Dropout(0.15),                    # Регуляризация
            nn.Linear(emb_dim // 2, num_classes) # Финальный слой
        )
        
    def forward(self, text_input, image_input):
        text_emb, image_emb = self.base_model(text_input, image_input)
        
        text_emb = text_emb.unsqueeze(0)  
        image_emb = image_emb.unsqueeze(0)  
        
        attended, _ = self.cross_attn(
            query=text_emb,
            key=image_emb,
            value=image_emb
        )

        # логиты предсказаний
        logits = self.classifier(attended.squeeze(0))
        return logits 

# Реализация пайплайна обучения мультимодальной нейросети и замера качества

## Пайплайн обучения мультимодальной нейросети

Обучение мультимодальной модели мало чем отличается от знакомых вам одномодальных архитектур. Вам нужно:

- Создать загрузчики и обёртки для датасетов.
    
- Сформировать модуль с архитектурой модели.
    
- Реализовать цикл обучения и валидации.

Кроме данных и итоговой модели, для экспериментов необходим конфиг запуска — набор параметров, описывающих архитектуру, гиперпараметры и обработку данных.

Существует бесконечное множество форматов представления данных файлов конфигурации — от JSON-файла до Python-класса.

В качестве примера реализуем свой конфиг Config для запуска обучения через Python-класс. Передадим в него ключевые параметры, определённые в предыдущих уроках, и добавим полезные настройки:

- имена моделей;
    
- размер батча;
    
- число эпох обучения;
    
- количество классов в задаче (если мы говорим о задаче классификации);
    
- численные значения, определяющие обучения PyTorch модулей (например, dropout);
    
- размерность эмбеддингов для проекции;
    
- learning rate (LR).

Отдельно отметим нюансы, связанные со скоростью обучения.
При тюнинге LR обычно ниже, чем при обучении с нуля. У разных архитектур свои оптимальные значения: трансформеры для NLP часто требуют LR на порядок меньше, чем CV архитектуры вроде EfficientNet. Для MLP‑головы тоже можно варьировать LR.

## 1.Построение универсального конфига.

In [ ]:
class Config:
    # Модели
    TEXT_MODEL_NAME = "bert-base-uncased"
    IMAGE_MODEL_NAME = "resnet50"
    
    # Какие слои размораживаем - совпадают с неймингом в моделях
    TEXT_MODEL_UNFREEZE = "encoder.layer.11|pooler"  
    IMAGE_MODEL_UNFREEZE = "layer.3|layer.4"  
    
    # Гиперпараметры
    BATCH_SIZE = 32
    TEXT_LR = 3e-5
    IMAGE_LR = 1e-4
    CLASSIFIER_LR = 5e-4
    EPOCHS = 10
    DROPOUT = 0.3
    HIDDEN_DIM = 256
    NUM_CLASSES = 5
    
    # Пути
    TRAIN_DF_PATH = "path/train.csv"
    VAL_DF_PATH = "path/val.csv"
    SAVE_PATH = "best_model.pth" 

Оптимизаторы `PyTorch` могут одновременно обучать разные параметры модели с разными настройками. Например, для абстрактной модели `model`, состоящей из двух частей — `embedder` и `classifier`, можно передать в любой оптимизатор `PyTorch` настройки params (веса подмодуля) и `lr` (`learning rate`, используемый для этих весов)

```py
# на примере оптимизатора AdamW

optimizer = AdamW([
    {'params': model.embedder.parameters(), 'lr': 0.001},
    {'params': model.classifier.parameters(), 'lr': 0.0001},
])
```

Кроме того, не всегда нужно обучать всю модель с нуля — иногда достаточно заморозить часть слоёв или, наоборот, всю модель, оставив для обучения только классификатор. 

Набор слоёв для разморозки можно указать в конфиге модели текстовым полем (предварительно стоит вывести структуру модели и выбрать слои по их именам). Обычно при частичной разморозке выбирают слои, идущие подряд — с первого по последний без пропусков. Например, в трёхслойной сети нельзя разморозить только первый и третий слои.

Для удобства разморозки можно объединить этот процесс в функцию set_requires_grad, которая принимает модель (или её часть, nn.Module) и unfreeze_pattern — текстовые имена слоёв для разморозки (или несколько имён, разделённых |, как в Config выше, например: "layer.3|layer.4" ).

Итерируясь по параметрам и их именам, функция размораживает выбранные слои. 

In [2]:
import timm


def set_requires_grad(module, unfreeze_pattern, verbose=False):
    # Если пустая строка - замораживаем все
    if len(unfreeze_pattern) == 0:
        for _, param in module.named_parameters():
            param.requires_grad = False
        return

    # разбиваем все слои
    pattern = unfreeze_pattern.split("|")

    # Проходим по всем слоям и ищем совпадения любого
    # слоя из `pattern` с текущим именем слоя
    for name, param in module.named_parameters():
        if any([name.startswith(p) for p in pattern]):
            param.requires_grad = True
            if verbose:
                print(f"Разморожен слой: {name}")
        else:
            param.requires_grad = False


# Разморозка последнего слоя resnet50
image_model = timm.create_model(
            'resnet50',
            pretrained=True,
            num_classes=0
        )

set_requires_grad(image_model, unfreeze_pattern="layer4", verbose=True) 

Разморожен слой: layer4.0.conv1.weight
Разморожен слой: layer4.0.bn1.weight
Разморожен слой: layer4.0.bn1.bias
Разморожен слой: layer4.0.conv2.weight
Разморожен слой: layer4.0.bn2.weight
Разморожен слой: layer4.0.bn2.bias
Разморожен слой: layer4.0.conv3.weight
Разморожен слой: layer4.0.bn3.weight
Разморожен слой: layer4.0.bn3.bias
Разморожен слой: layer4.0.downsample.0.weight
Разморожен слой: layer4.0.downsample.1.weight
Разморожен слой: layer4.0.downsample.1.bias
Разморожен слой: layer4.1.conv1.weight
Разморожен слой: layer4.1.bn1.weight
Разморожен слой: layer4.1.bn1.bias
Разморожен слой: layer4.1.conv2.weight
Разморожен слой: layer4.1.bn2.weight
Разморожен слой: layer4.1.bn2.bias
Разморожен слой: layer4.1.conv3.weight
Разморожен слой: layer4.1.bn3.weight
Разморожен слой: layer4.1.bn3.bias
Разморожен слой: layer4.2.conv1.weight
Разморожен слой: layer4.2.bn1.weight
Разморожен слой: layer4.2.bn1.bias
Разморожен слой: layer4.2.conv2.weight
Разморожен слой: layer4.2.bn2.weight
Разморожен 

Для метрик достаточно полезно использовать torchmetrics 
Важно перенести метрику на верный device.

```py

import torchmetrics

# порог по умолчанию равен 0.5
device = "cuda" if torch.cuda.is_available() else "cpu"
f1 = torchmetrics.F1Score(task="binary", num_classes=2).to(device)
```

Чтобы вычислить метрику и сохранить промежуточный результат, необходимо передать в неё тензоры с предсказаниями и истинные метки объектов. 

Чтобы агрегировать результаты между батчами, необходимо вызвать метод compute(), чтобы обнулить сохранённые результаты — метод reset().

```py
# обнуляем метрику, можно вызывать в любой момент
f1.reset()

for i in [0, 1, 1, 0]:
    # имимтируем разные метрики в батчах
    score = f1(preds=torch.Tensor([0.1, 0.6, i, 0.2]),
               target=torch.Tensor([0, 1, 1, 1]))
    print(score)

# агрегируем результаты
score = f1.compute()
print(score)
```

```py
batch score: tensor(0.5000) 
batch score: tensor(0.8000) 
batch score: tensor(0.8000) 
batch score: tensor(0.5000) 
epoch score: tensor(0.6667)
```

Мультимодальные модели с точки зрения кода не сильно отличаются от обычных одномодальных архитектур. Основные отличия заключаются в необходимости слияния признаков разных модальностей, а также подбора качественного мультимодального датасета. 

В задачах классификации становится возможным дополнительно оценить влияние каждой из модальностей на ответ модели, путём маскирования смежной.

Подведём итог всей темы. Вы проделали большой путь: от подготовки данных и аугментаций до сбора мультимодального датасета и обучения модели, которая может обрабатывать картинки и тексты одновременно. 

На реальном кейсе разобрались в графиках обучения моделей и сделали выводы о том, как можно оценивать влияние различных признаков на целевую задачу.

Эти знания — крепкая основа для решения базовых задач Deep Learning.